# k-Nearest Neighbors Classifier

In this notebook, I've implement my own k-nearest neighbors (k-NN) algorithm for a classification problem using only Pandas and Numpy as additional packages.

The dataset, mushrooms.csv,  includes characteristics/attributes of mushrooms. I will implement k-nearest neighbors classifier, and using it to say whether a mushroom is poisonous or edible based on its attributes.
The dataset of mushroom characteristics is freely available at [Kaggle Datasets](https://www.kaggle.com/uciml/mushroom-classification)  It consists of 8124 mushrooms characterized by 23 attributes (including the class).

In [1]:
import pandas as pd
import numpy as np

Let's load the dataset into so called Pandas dataframe.

In [2]:
mushrooms_df = pd.read_csv('mushrooms.csv')

Now we can take a closer look at the data.

In [3]:
mushrooms_df

,class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,p,x,s,n,t,p,f,c,n,k,...,s,w,w,p,w,o,p,k,s,u
1,e,x,s,y,t,a,f,c,b,k,...,s,w,w,p,w,o,p,n,n,g
2,e,b,s,w,t,l,f,c,b,n,...,s,w,w,p,w,o,p,n,n,m
3,p,x,y,w,t,p,f,c,n,n,...,s,w,w,p,w,o,p,k,s,u
4,e,x,s,g,f,n,f,w,b,k,...,s,w,w,p,w,o,e,n,a,g
5,e,x,y,y,t,a,f,c,b,n,...,s,w,w,p,w,o,p,k,n,g
6,e,b,s,w,t,a,f,c,b,g,...,s,w,w,p,w,o,p,k,n,m
7,e,b,y,w,t,l,f,c,b,n,...,s,w,w,p,w,o,p,n,s,m
8,p,x,y,w,t,p,f,c,n,p,...,s,w,w,p,w,o,p,k,v,g
9,e,b,s,y,t,a,f,c,b,g,...,s,w,w,p,w,o,p,k,s,m


## Dataset Preprocessing

As our dataset consist of nominal/categorical values only, we will encode the strings into integers which will allow us to use similiraty measures such as Euclidean distance.

In [4]:
def encode_labels(df):
    import sklearn.preprocessing
    encoder = {}
    for col in df.columns:
        le = sklearn.preprocessing.LabelEncoder()
        le.fit(df[col])
        df[col] = le.transform(df[col])
        encoder[col] = le
    return df, encoder    

mushrooms_encoded_df, encoder = encode_labels(mushrooms_df)

In [5]:
mushrooms_encoded_df

,class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,1,5,2,4,1,6,1,0,1,4,...,2,7,7,0,2,1,4,2,3,5
1,0,5,2,9,1,0,1,0,0,4,...,2,7,7,0,2,1,4,3,2,1
2,0,0,2,8,1,3,1,0,0,5,...,2,7,7,0,2,1,4,3,2,3
3,1,5,3,8,1,6,1,0,1,5,...,2,7,7,0,2,1,4,2,3,5
4,0,5,2,3,0,5,1,1,0,4,...,2,7,7,0,2,1,0,3,0,1
5,0,5,3,9,1,0,1,0,0,5,...,2,7,7,0,2,1,4,2,2,1
6,0,0,2,8,1,0,1,0,0,2,...,2,7,7,0,2,1,4,2,2,3
7,0,0,3,8,1,3,1,0,0,5,...,2,7,7,0,2,1,4,3,3,3
8,1,5,3,8,1,6,1,0,1,7,...,2,7,7,0,2,1,4,2,4,1
9,0,0,2,9,1,0,1,0,0,2,...,2,7,7,0,2,1,4,2,3,3


## Dataset Splitting

Before we start with the implementation of our k-nearest neighbors algorithm we need to prepare our dataset for the "training" and testing.

First, we divide the dataset into attributes (often called features) and classes (often called targets). This should simplify the implementation and make the code understandable.

In [7]:
X_df = mushrooms_encoded_df.drop('class', axis=1)  # attributes
y_df = mushrooms_encoded_df['class']  # classes
X_array = X_df.as_matrix()
y_array = y_df.as_matrix()

C:\Users\larsb\Anaconda\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\larsb\Anaconda\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  after removing the cwd from sys.path.


And this is how it looks like.

In [8]:
print('X =', X_array)
print('y =', y_array)

X = [[5 2 4 ... 2 3 5]
 [5 2 9 ... 3 2 1]
 [0 2 8 ... 3 2 3]
 ...
 [2 2 4 ... 0 1 2]
 [3 3 4 ... 7 4 2]
 [5 2 4 ... 4 1 2]]
y = [1 0 0 ... 0 1 0]


Next, we need to split the attributes and classes into training sets and test sets.

In [80]:
import random
from math import floor
def train_test_split(X, y, test_size=0.2):
    """
    Shuffles the dataset and splits it into training and test sets.
    
    :param X
        attributes
    :param y
        classes
    :param test_size
        float between 0.0 and 1.0 representing the proportion of the dataset to include in the test split
    :return
        train-test splits (X-train, X-test, y-train, y-test)
    """
    #Random shuffels X and y
    #Adding y as last col in X beforw shuffeling
    X = X.tolist()
    for i in range(len(X)):
        X[i].append(y[i])
        
    #shuffels    
    random.shuffle(X) 
    
    #set y equal las col in X
    y = [X[i][-1] for i in range(len(X))]
    
    #delete last col in X
    for i in range(len(X)):
        X[i].pop()
    
    #Split X into X_test, y_test, x_train, y_train
    X_test = X[0:floor(len(X)*test_size)]
    y_test = y[0:floor(len(y)*test_size)]
   
    y_train = y[len(y_test): ]
    X_train = X[len(X_test): ]
    

    return X_train, X_test, y_train, y_test

Let's split the dataset into training and validation/test set with 67:33 split.

In [81]:
X_train, X_test, y_train, y_test = train_test_split(X_array, y_array, 0.33)

In [82]:
print('X_train =', np.array(X_train))
print('y_train =', np.array(y_train))
print('X_test =', np.array(X_test))
print('y_test =', np.array(y_test))

X_train = [[5 3 3 ... 3 5 0]
 [3 2 4 ... 7 4 4]
 [0 2 8 ... 7 2 1]
 ...
 [3 2 2 ... 7 4 0]
 [2 2 8 ... 1 3 1]
 [5 3 2 ... 7 4 0]]
y_train = [0 1 0 ... 1 1 1]
X_test = [[5 3 4 ... 7 4 4]
 [5 0 3 ... 1 4 4]
 [2 0 3 ... 1 5 1]
 ...
 [2 3 3 ... 1 5 4]
 [5 3 3 ... 1 5 0]
 [5 3 3 ... 1 4 0]]
y_test = [1 1 1 ... 1 1 1]


A quick sanity check...

In [83]:
assert len(X_train) == len(y_train)
assert len(y_train) == 5444
assert len(X_test) == len(y_test)
assert len(y_test) == 2680

## Algorithm

The k-nearest neighbors algorithm doesn't require a training step. The class of an unseen sample is deduced by comparison with samples of known class.


Implementing the k-nearest neighbors algorithm:

In [96]:
import math
import operator 


#to calc the euc_dist between to shrooms. length is the of the testingset
def euclideanDistance(shroom1, shroom2, length):
    distance = 0
    for x in range(length):
        distance += pow((shroom1[x] - shroom2[x]), 2)
    return math.sqrt(distance)

#t find the k nearest neighboors of a shroom
def getKNeighbors(trainingSet, shroom, k):
    distances = []
    length = len(shroom)
    for x in range(len(trainingSet)):
        dist = euclideanDistance(shroom, trainingSet[x], length)
        distances.append((trainingSet[x], dist))
    distances.sort(key=operator.itemgetter(1))
    neighbors = []
    for x in range(k):
        neighbors.append(distances[x][0])
    return neighbors


#a "voting-function". Looks at the classes of all naighbors and return the most common one.
def getResponse(neighbors):
    classVotes = {}
    for x in range(len(neighbors)):
        clas = neighbors[x]
        if clas in classVotes:
            classVotes[clas] +=1
        else:
            classVotes[clas] = 1
    sortedVotes = sorted(classVotes.items(), key=operator.itemgetter(1), reverse=True)
    return sortedVotes[0][0]



In [98]:
def knn(X_true, y_true, X_pred, k=5):
    
    #Finds the k nearest neighbors for each shroom
    all_K_neighbors = []
    for x in range(len(X_pred)):
        all_K_neighbors.append(getKNeighbors(X_true, X_pred[x], k))
    
    #find the relative class-values
    K_neighbors_y = []
    for n in all_K_neighbors:
        K_neighbors_y.append([])
        for m in n:
            K_neighbors_y[-1].append(y_true[X_true.index(m)])
    
    #makes the predictions by looking at the nearest neighbors
    y_pred = []
    for ys in K_neighbors_y:
        y_pred.append(getResponse(ys))

    return y_pred

In [97]:
y_hat = knn(X_train, y_train, X_test, k=5)

First ten predictions of the test set.

In [91]:
y_hat[:10]

[1, 1, 1, 0, 1, 1, 0, 0, 0, 1]

## Evaluation

Now we would like to assess how well our classifier performs.

In [99]:
def evaluate(y_true, y_pred):
    correct = 0
    for x in range(len(y_true)):
        if y_true[x] == y_pred[x]:
            correct += 1
            
    accuracy = (correct/float(len(y_true))) * 100.0

    return accuracy

In [100]:
accuracy = evaluate(y_test, y_hat)
print('accuracy =', accuracy)

accuracy = 99.92537313432835


How many items where misclassified?

In [101]:
print('misclassified =', sum(abs(np.array(y_hat) - np.array(y_test))))

misclassified = 2


How balanced is our test set?

In [95]:
np.bincount(y_test)

array([1395, 1285], dtype=int64)

If it's balanced, we don't have to be worried about objectivity of the accuracy metric.